
![Universidad Central](https://www.ucentral.edu.co/sites/default/files/inline-images/identificadores-Centrados_en_ti_logo-h.png)

**Tarea numero 4**

**Predicción del Valor de Venta de Inmuebles***

**Metodos Estadisticos** <br>

Profesor: Docente: Luis Andrés Campos Maldonado <br>
Facultad de Ingeniería y Ciencias Básicas <br>
Universidad Central <br>

__GRUPO 4__ <br>
- Daniel Mauricio Cardenas
- Andrea Catalina Gomez
- Luisa Fernanda Rodriguez

# Introducción
En este ejercicio, analizarás un conjunto de datos de inmuebles para predecir el valor de venta (`valor_venta`) en función de sus características. Utilizarás tres modelos
- **Regresión Lineal**
- **Árbol de Decisión**
- **Random Forest**

 El proceso incluirá limpieza de datos, análisis exploratorio y evaluación del desempeño de los modelos.

---

## Descripción de las columnas
1. **`tipo_inmueble`**: Tipo de propiedad (e.g., Apartamento, Casa).
2. **`valor_venta`**: Valor de venta del inmueble en pesos colombianos.
3. **`area`**: Área del inmueble en metros cuadrados.
4. **`nro_cuartos`**: Número de cuartos en la propiedad.
5. **`nro_banos`**: Número de baños en la propiedad.
6. **`nro_garajes`**: Número de garajes disponibles.
7. **`zona`**: Zona geográfica donde se encuentra el inmueble (e.g., Chapinero, Norte).
8. **`barrio`**: Nombre específico del barrio.
9. **`nombre_comun_barrio`**: Nombre común asociado al barrio.

---

El ejercicio debe cubrir los siguientes pasos:

***Parte 1: Limpieza de datos***
1. Ajusta el tipo de dato de cada columna según su naturaleza (numérica o categórica).
2. Maneja valores nulos imputándolos.

***Parte 2: Análisis exploratorio de datos (EDA)***
1. Genera estadísticas descriptivas de las variables.
2. Visualiza distribuciones y relaciones entre variables predictoras y la variable objetivo.
3. Analiza cómo cambian las categorías en relación con la variable objetivo.

***Parte 3: Construcción y evaluación de modelos***
1. Divide los datos en conjuntos de entrenamiento (80%) y prueba (20%).
2. Construye y ajusta los siguientes modelos:
   - **Regresión Lineal**.
   - **Árbol de Decisión**.
   - **Random Forest**.
3. Evalúa el desempeño de los modelos utilizando métricas como **MAE**, **MAPE**, **RMSE**.
4. Realiza un análisis residual para diagnosticar posibles problemas.


***Notas***
- Justifica todas las decisiones tomadas.
- Identifica y analiza el impacto de valores atípicos.
- Si deseas puedes hacer transformacion de variables (es necesario para todos los modelos?)

In [1]:
import pandas as pd
data_tarea4 = pd.read_csv("https://raw.githubusercontent.com/lacamposm/Metodos-Estadisticos/main/data/tarea4_regresion.csv")

In [2]:
# Librerias a utilizar

import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split            # Train, Test sets
from sklearn.metrics import mean_absolute_error, r2_score       # Para obtener las métricas de evaluación.
from sklearn.metrics import mean_absolute_percentage_error      # Para obtener las métricas de evaluación.
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.tree  import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    confusion_matrix,
    accuracy_score,
    roc_curve,
    auc
)

# Parte estadística.
import statsmodels.formula.api  as smf                          # Modelos lineal.
from scipy.stats import shapiro, probplot, f, f_oneway          # libreria estadistica de Scipy
from statsmodels.stats.anova import anova_lm                    # Anova del modelo lineal

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (15,6)
warnings.filterwarnings("ignore")

# Limpieza de los datos

Se crea una copia del Dataset original, validamos los tipos de datos para cada una de las columnas. 

In [3]:
data_clean = data_tarea4.copy()
print(f'{data_clean.info()}') # Información del Dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tipo_inmueble        10000 non-null  object 
 1   valor_venta          10000 non-null  int64  
 2   area                 10000 non-null  float64
 3   nro_cuartos          9993 non-null   object 
 4   nro_banos            10000 non-null  object 
 5   nro_garajes          9945 non-null   float64
 6   zona                 9812 non-null   object 
 7   barrio               9945 non-null   object 
 8   nombre_comun_barrio  9866 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 703.2+ KB
None


Se identifica que las columnas 'nro_cuartos' y 'nro_baños', son variables categoricas (objet), estas se cambian a numericas (float)

In [4]:
display(
    data_clean['nro_cuartos'].value_counts(),
    data_clean['nro_banos'].value_counts()
)

nro_cuartos
3     5043
2     1979
4     1534
1      855
5      576
5+       6
Name: count, dtype: int64

nro_banos
2     3276
3     2735
4     1726
5     1470
1      659
0      122
5+      12
Name: count, dtype: int64

In [5]:
data_clean['nro_cuartos'] = data_clean['nro_cuartos'].str.replace('+','',case = False,regex=False)
data_clean['nro_banos'] = data_clean['nro_banos'].str.replace('+','',case = False,regex=False)
display(
    data_clean['nro_cuartos'].value_counts(),
    data_clean['nro_banos'].value_counts()
)

nro_cuartos
3    5043
2    1979
4    1534
1     855
5     582
Name: count, dtype: int64

nro_banos
2    3276
3    2735
4    1726
5    1482
1     659
0     122
Name: count, dtype: int64

In [6]:
data_clean[['nro_cuartos','nro_banos']] = data_clean[['nro_cuartos','nro_banos']].astype('float')

In [7]:
print (f'{data_clean.isnull().sum()}') # Valores nulos

tipo_inmueble            0
valor_venta              0
area                     0
nro_cuartos              7
nro_banos                0
nro_garajes             55
zona                   188
barrio                  55
nombre_comun_barrio    134
dtype: int64


Ahora, ideintificamos los valores nulos para cada una de las columnas.

In [8]:
data_clean[data_clean['nro_cuartos'].isna()]

,tipo_inmueble,valor_venta,area,nro_cuartos,nro_banos,nro_garajes,zona,barrio,nombre_comun_barrio
4492,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
4702,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
4912,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5122,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5332,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5542,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5752,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico


In [9]:
data_clean[data_clean['area']== 30]

,tipo_inmueble,valor_venta,area,nro_cuartos,nro_banos,nro_garajes,zona,barrio,nombre_comun_barrio
141,Apartamento,200000000,30.0,1.0,1.0,1.0,Noroccidente,ESTORIL,PASADENA
351,Apartamento,200000000,30.0,1.0,1.0,1.0,Noroccidente,ESTORIL,PASADENA
561,Apartamento,200000000,30.0,1.0,1.0,1.0,Noroccidente,ESTORIL,PASADENA
771,Apartamento,200000000,30.0,1.0,1.0,1.0,Noroccidente,ESTORIL,PASADENA
4492,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
4702,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
4912,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5122,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5332,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico
5542,Apartamento,160000000,30.0,NaN,1.0,0.0,Norte,CHICO,El Chico


In [10]:
data_clean['nro_cuartos'] = data_clean['nro_cuartos'].fillna(1)

Para la columna 'nro_cuartos', identificamos que los valores nulos corresponden exclusivamente a apartamentos con un área de 30 m², ubicados en la zona norte. Tras analizar el patrón en apartamentos con características similares (mismo tamaño y ubicación), observamos que típicamente tienen 1 cuarto. Por lo tanto, decidimos imputar los valores nulos en esta columna con el valor 1, asegurando coherencia con el comportamiento de los datos existentes.

In [11]:
data_clean[data_clean['nro_garajes'].isna()]

,tipo_inmueble,valor_venta,area,nro_cuartos,nro_banos,nro_garajes,zona,barrio,nombre_comun_barrio
238,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
448,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
658,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
868,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
1078,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
1235,Apartamento,760000000,102.0,2.0,3.0,NaN,Norte,CHICO,CHICO NORTE
1288,Apartamento,960000000,135.0,3.0,3.0,NaN,Chapinero,LAS ACACIAS,LOS ROSALES
1445,Apartamento,760000000,102.0,2.0,3.0,NaN,Norte,CHICO,CHICO NORTE
1655,Apartamento,760000000,102.0,2.0,3.0,NaN,Norte,CHICO,CHICO NORTE
1865,Apartamento,760000000,102.0,2.0,3.0,NaN,Norte,CHICO,CHICO NORTE


In [12]:
garajes = data_clean.groupby(['zona']).agg(nro_garajes_mean=('nro_garajes', 'mean')).reset_index()
garajes

,zona,nro_garajes_mean
0,Centro,0.809756
1,Chapinero,1.382637
2,Guaymaral,2.924528
3,Noroccidente,2.031095
4,Norte,2.052548
5,Occidental,0.955796
6,Otros,1.671053
7,Sur,0.805970


In [ ]:
data_clean.loc[(data_clean['zona'] == 'Chapinero') & (data_clean['nro_garajes'].isnull()), 'nro_garajes'] = 1
data_clean.loc[(data_clean['zona'] == 'Norte') & (data_clean['nro_garajes'].isnull()), 'nro_garajes'] = 2
data_clean.loc[(data_clean['zona'] == 'Sur') & (data_clean['nro_garajes'].isnull()), 'nro_garajes'] = 1

nro_garajes
2.0    3959
1.0    2652
0.0    1191
3.0    1178
4.0    1020
Name: count, dtype: int64

nro_garajes
2.0    3916
1.0    2640
0.0    1191
3.0    1178
4.0    1020
Name: count, dtype: int64

,tipo_inmueble,valor_venta,area,nro_cuartos,nro_banos,nro_garajes,zona,barrio,nombre_comun_barrio


Dado que no se puede identificar un patrón claro para los valores faltantes en la columna 'nro_garajes', analizamos las zonas afectadas (Chapinero, Norte, y Sur) y calculamos el promedio de garajes para cada una. Luego, imputamos los valores faltantes con el promedio correspondiente a cada zona, redondeado al entero más cercano. Esto garantiza que la imputación sea coherente con las características promedio de cada zona.

In [14]:
data_clean[['zona','barrio','nombre_comun_barrio']] = data_clean[['zona','barrio','nombre_comun_barrio']].fillna('Desconocido')

Por ultimo para las variables categoricas 'zona','barrio','nombre_comun_barrio', se imputa a los valores nulos como Desconicido, para manejar categoria inexsistentes sin eliminar información util.

In [15]:
data_clean.info()
print (f'{data_clean.isnull().sum()}') # Valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tipo_inmueble        10000 non-null  object 
 1   valor_venta          10000 non-null  int64  
 2   area                 10000 non-null  float64
 3   nro_cuartos          10000 non-null  float64
 4   nro_banos            10000 non-null  float64
 5   nro_garajes          10000 non-null  float64
 6   zona                 10000 non-null  object 
 7   barrio               10000 non-null  object 
 8   nombre_comun_barrio  10000 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 703.2+ KB
tipo_inmueble          0
valor_venta            0
area                   0
nro_cuartos            0
nro_banos              0
nro_garajes            0
zona                   0
barrio                 0
nombre_comun_barrio    0
dtype: int64


# AED